In [ ]:
!pip install tensorflow
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Layer
from google.colab import files
from tensorflow.keras import backend as K
print("✅ Libraries imported")


✅ Libraries imported


In [ ]:
df = pd.read_csv('output.csv')

In [ ]:
df.head()

,er,fr
0,Let's try something.,Tentons quelque chose !
1,I have to go to sleep.,Je dois aller dormir.
2,Today is June 18th and it is Muiriel's birthday!,Aujourd'hui nous sommes le 18 juin et c'est l'...
3,Today is June 18th and it is Muiriel's birthday!,"Aujourd'hui c'est le 18 juin, et c'est l'anniv..."
4,Muiriel is 20 now.,Muiriel a 20 ans maintenant.


In [ ]:
src_texts = df['er'].astype(str).tolist()
tgt_texts = df['fr'].astype(str).tolist()

In [ ]:
tgt_input_texts = ['<sos> ' + text for text in tgt_texts]
tgt_target_texts = [text + ' <eos>' for text in tgt_texts]

In [ ]:
# Tokenize source
src_tokenizer = Tokenizer()
src_tokenizer.fit_on_texts(src_texts)
src_seq = src_tokenizer.texts_to_sequences(src_texts)
src_seq = pad_sequences(src_seq, padding='post')

tgt_tokenizer = Tokenizer(filters='')
tgt_tokenizer.fit_on_texts(tgt_input_texts + tgt_target_texts)
tgt_input_seq = pad_sequences(tgt_tokenizer.texts_to_sequences(tgt_input_texts), padding='post')
tgt_output_seq = pad_sequences(tgt_tokenizer.texts_to_sequences(tgt_target_texts), padding='post')


src_vocab_size = len(src_tokenizer.word_index) + 1
tgt_vocab_size = len(tgt_tokenizer.word_index) + 1


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate

latent_dim = 256

encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(input_dim=src_vocab_size, output_dim=latent_dim, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(latent_dim, return_sequences=True, return_state=True)(enc_emb)

decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(input_dim=tgt_vocab_size, output_dim=latent_dim, mask_zero=True)(decoder_inputs)
decoder_outputs, _, _ = LSTM(latent_dim, return_sequences=True, return_state=True)(
    dec_emb, initial_state=[state_h, state_c]
)


In [ ]:

attention_layer = Attention()
context_vector = Attention()(
    [decoder_outputs, encoder_outputs]
)

combined = Concatenate(axis=-1)([decoder_outputs, context_vector])  # shape: (batch, tgt_len, latent_dim*2)

output = Dense(tgt_vocab_size, activation='softmax')(combined)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], output)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_25      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_26      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_24        │ (None, None, 256) │  1,457,152 │ input_layer_25[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_24        │ (None, None)      │          0 │ input_layer_25[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_25        │ (None, None, 256) │  3,103,232 │ input_layer_26[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_24 (LSTM)      │ [(None, None,     │    525,312 │ embedding_24[0][… │
│                     │ 256), (None,      │            │ not_equal_24[0][… │
│                     │ 256), (None,      │            │                   │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_25 (LSTM)      │ [(None, None,     │    525,312 │ embedding_25[0][… │
│                     │ 256), (None,      │            │ lstm_24[0][1],    │
│                     │ 256), (None,      │            │ lstm_24[0][2]     │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_25        │ (None, None)      │          0 │ input_layer_26[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_6         │ (None, None, 256) │          0 │ lstm_25[0][0],    │
│ (Attention)         │                   │            │ lstm_24[0][0],    │
│                     │                   │            │ not_equal_25[0][… │
│                     │                   │            │ not_equal_24[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_26      │ (None, None, 512) │          0 │ lstm_25[0][0],    │
│ (Concatenate)       │                   │            │ attention_6[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, None,      │  6,218,586 │ concatenate_26[0… │
│                     │ 12122)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,829,594 (45.13 MB)

 Trainable params: 11,829,594 (45.13 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(
    [src_seq, tgt_input_seq],
    tgt_output_seq[..., None],
    batch_size=64,
    epochs=10,
    validation_split=0.2
)
